## Setup

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.distance import cdist
from google.colab.patches import cv2_imshow

In [2]:
from os import listdir
from os.path import isfile, join
import glob

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Cropping original frames

In [ ]:
# video = '1606b0e6_0'

In [4]:
CLASS_INDEX = 0
X_INDEX = 1
Y_INDEX = 2
PERSON_CLASS = '0'
BALL_CLASS = '32'
X_SIZE = 1928
Y_SIZE = 1024
CROP_SIZE = 224 # pixels, size of crop = (CROP_SIZExCROP_SIZE)
radius = int(CROP_SIZE / 2)
ANY_INDEX = 0

In [5]:
# files = [file for file in listdir(FOLDER_FRAMES) if isfile(join(FOLDER_FRAMES, file))] # get a list of files in the directory
# filenames = [os.path.splitext(filename)[0] for filename in files] # separate filenames from their extension .jpg

## Test: doing above by iterating over 2 different folders (frames and labels)

In [6]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
FOLDER_LABELS = POSITIVES_FOLDER + 'detect/'
FOLDER_FRAMES = POSITIVES_FOLDER + 'frames_ms2/'

FIRST_FRAMES = FOLDER_FRAMES + '1606b0e6_0/'
SECOND_FRAMES = FOLDER_FRAMES + '1606b0e6_1/'

In [7]:
CURRENT_FOLDER = POSITIVES_FOLDER
os.chdir(CURRENT_FOLDER)
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


## Main code:

In [8]:
# video = "_".join(filenames[ANY_INDEX].split("_", 2)[:2])
# rest = ''.join(filenames[ANY_INDEX].split("_", 2)[2:])
# y = filenames[ANY_INDEX].split("_")[3]
# video, rest, y

In [9]:
# current_labels = FOLDER_LABELS + video + '/labels/'
# current_labels

In [201]:
USE_PERSONS = True
USE_PERSONS_THRESH = 15
FRAMES_PER_EVENT = 35
NUM_CLASSES = 3
CONFIDENCE_INDEX = 5

FRAMES_FORMAT = 'VIDEOID_EVENTID_CLASSID_ORDERID.jpg'

Class meanings = {0: 'challenge', 1: 'throw-in', 2: 'play'}

In [ ]:
# Challenges' frame between tolerances are respectively: 17 yes, 1 yes, 17 yes.
# For play and throw-in are respectively: 9 no, 8 yes, 1 yes, 8 yes, 9 no.

Implementation:

In [252]:
video_ids = [
    # '1606b0e6_0', 
            #  '1606b0e6_1',
             'cfbe2e94_0', 
             'cfbe2e94_1',
            #  'ecf251d4_0',
            #  '3c993bd2_0', 
            #  '3c993bd2_1',
             '4ffd5986_0', 
             '9a97dae4_1', 
            #  '35bd9041_0', 
            #  '35bd9041_1',
            #  '407c5a9e_1',
             ]

In [255]:
EVENTS_PER_VIDEO = {
    # '1606b0e6_0': 396,
                    # '1606b0e6_1':,
                    'cfbe2e94_0': 305,
                    'cfbe2e94_1': 285,
                    # 'ecf251d4_0': 386,
                    # '3c993bd2_1': 376,
                    # '3c993bd2_1':,
                    '4ffd5986_0': 302,
                    '9a97dae4_1': 291,
                    # '35bd9041_0':, 
                    # '35bd9041_1':,
                    # '407c5a9e_1': 350,
                    }
# MAYBE CODE IT IN A BETTER WAY

In [15]:
# ! cat frames_ms2/3c993bd2_1 | find "3c993bd2_1_105" # DOESN'T WORK

In [16]:
# ! grep "3c993bd2_1_105" frames_ms2/3c993bd2_1 # DOESN'T WORK

In [17]:
# ! ls frames_ms2/3c993bd2_1

In [18]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
FOLDER_LABELS = POSITIVES_FOLDER + 'detect/'
FOLDER_FRAMES = POSITIVES_FOLDER + 'frames_ms2/'

FIRST_FRAMES = FOLDER_FRAMES + '1606b0e6_0/'
SECOND_FRAMES = FOLDER_FRAMES + '1606b0e6_1/'

In [198]:
os.chdir(POSITIVES_FOLDER)
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


In [20]:
# ! mkdir crops

In [21]:
FIRST_FRAME_USED = 18

In [22]:
FOLDER_LABELS

'/content/drive/MyDrive/Bundesliga/PositiveFrames/detect/'

In [256]:
videos = [video for video in EVENTS_PER_VIDEO.keys()]
videos

['cfbe2e94_0', 'cfbe2e94_1', '4ffd5986_0', '9a97dae4_1']

## Main code:

### Hyperparameter tunning to find best threshold for distance ball-person

In [32]:
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


In [226]:
video = '1606b0e6_0'
event = 0
class_ = 0
r = 1

In [227]:
from scipy import spatial

In [228]:
event_id = f'{video}_{event}_{class_}'

frame_id = f'{event_id}_{FIRST_FRAME_USED - 1 + r}' # -1 so we get main fraim (index 17)
label_txt = f'{frame_id}.txt'
with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
  reader = f.read()
detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"

arr = np.array(detections)
persons = arr[arr[:, 0] == '0']
xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
detection = arr[arr[:, 0] == '32'] # DON'T USE IN ACTUAL CODE, YOU ALREADY HAVE 'DETECTION'
xy_ball = detection[:, X_INDEX:Y_INDEX + 1].astype(float)
xy_closest_person = xy_persons[spatial.KDTree(xy_persons).query(xy_ball)[1]] # get closest point
distance, index = spatial.KDTree(xy_persons).query(xy_ball) # get distance
# distance 1606b0e6_0_0_0_17: 0.01715389 # ball confused with football boot
# distance 1606b0e6_0_1_0_17: 0.02588498 # ball behind goalkeeper
# distance 1606b0e6_0_2_1_17: 0.05881961 # ball behind goalkeeper, and 0.13542185 ball on court's side
# distance 1606b0e6_0_3_2_20: 0.01331324 # BALL ON PLAY

# distance 1606b0e6_0_4_2_19: 0.05413774 # BALL ON PLAY

# distance 1606b0e6_0_5_2_17: 0.22416412 # ball on top right court's side
# distance 1606b0e6_0_5_2_19: 0.22460785 # ball on top right court's side
# distance 1606b0e6_0_5_2_20: 0.22483156 # ball on top right court's side
# distance 1606b0e6_0_5_2_21: 0.22660578 # ball on top right court's side, and 0.01887572 BALL ON PLAY

# distance 1606b0e6_0_6_2_17: 0.21772145 # ball on top right court's side
# distance 1606b0e6_0_6_2_18: 0.21834437 # ball on top right court's side, and 0.01998301 BALL ON PLAY

# distance 1606b0e6_0_7_2_17: 0.02071127 # BALL ON PLAY

# distance 1606b0e6_0_8_2_24: 0.01237615 # ball confused with football boot

# distance 1606b0e6_0_9_2_21: 0.05960312 # BALL ON PLAY

# distance 1606b0e6_0_10_2_17: 0.16937768 # ball confused with head of person top bottom
# distance 1606b0e6_0_10_2_18: 0.1684855 # ball confused with head of person top bottom
# distance 1606b0e6_0_10_2_19: 0.16789392 # ball confused with head of person top bottom
# distance 1606b0e6_0_10_2_25: 0.04449379 # BALL ON PLAY

# distance 1606b0e6_0_10_2_17: 0.238802 # ball on top right court's side
# distance 1606b0e6_0_11_2_18: 0.23834602 # ball on top right court's side
# distance 1606b0e6_0_11_2_19: 0.16189149 # ball on top right court's side (same, person in crowd detected now)
# distance 1606b0e6_0_11_2_20: 0.06701084 # ball on top right court's side (same, another person detected now), and
# 0.0336005 BALL ON PLAY

# WE SET THRESHOLD TO 0.09 FOR NOW.

frame_id, distance, xy_ball

('1606b0e6_0_0_0_18', array([0.02767142]), array([[0.492448, 0.834259]]))

In [218]:
distance[0] > 0.08

False

### Code implemented

In [216]:
UNIQUE_INDEX = 0

In [233]:
ball_close_to_person = True
ball_person_threshold = 0.065 # AS STRICT AS I CAN BE, LOWER THAN 0.06 IS NOT OK
y_camera_threshold = 0.75

In [257]:
for video in videos:
  ! mkdir crops/$video
  # We iterate over every event, from 0 to 395
  for event in range(EVENTS_PER_VIDEO[video]):
    # We establish ball = False (in this event, ball wasn't yet recognized)
    ball = False
    # We should find for example f'1606b0e6_0_{frame}_' and establish that class_ equals what follows it

    # Sometimes fails for no reason and then works
    # it_worked = False
    # while not it_worked:
    #   try: 
    #     file = glob.glob(f'{substring}*.txt')[ANY_INDEX]
    #     substring = f'{FOLDER_LABELS}{video}/labels/{video}_{event}_'
    #     it_worked = True
    #   except IndexError:
    #     pass
    substring = f'{FOLDER_LABELS}{video}/labels/{video}_{event}_'
    file = glob.glob(f'{substring}*.txt')[ANY_INDEX]

    len_subs = len(substring)
    # print(substring)
    # print(len_subs)
    # print(file[len_subs])
    class_ = int(file[len_subs])

    event_id = f'{video}_{event}_{class_}'

    # We create a condition based on the class on how to iterate for the frames.
    # Challenges' frame between tolerances are respectively: 17 yes, 1 yes, 17 yes.
    # For play and throw-in are respectively: 9 no, 8 yes, 1 yes, 8 yes, 9 no.

    # if class_ == 0: # if it's a challenge
      
    # TO AVOID BALL FPs: we iterate over every frames, from 18 to 32
    for frame in range(FIRST_FRAME_USED, FRAMES_PER_EVENT - 2): # - 2 cause we need to use 2 more after the one detected
      frame_id = f'{event_id}_{frame}' # e.g. '1606b0e6_0_0_0_0'
      print(frame_id)
      label_txt = f'{frame_id}.txt'
      with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
        reader = f.read()
      detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"
      # counter = 0

      for detection in detections: # in a single frame

        # confidences = list() # WE ARE NOT IMPLEMENTING CONFIDENCES FOR NOW AS IT MAY NOT BE RELEVANT
        if detection[CLASS_INDEX] == BALL_CLASS:
          # Ball is True for this event, it was detected in at least one of the frames.
          ball = True
          
          # confidence = float(detection[CONFIDENCE_INDEX])

          x = int(float(detection[X_INDEX]) * X_SIZE)
          y = int(float(detection[Y_INDEX]) * Y_SIZE) # 0.75 * 1028 = 771
          ymin = max(y - radius, 0) # 771 - 112 = 659
          ymax = min(y + radius, Y_SIZE) # 771 + 112 = 883
          xmin = max(x - radius, 0)
          xmax = min(x + radius, X_SIZE)

          # TO AVOID BALL FPs: we throw away detections that make the crop next to a limit of the frame

          # CHANGED ymax == Y_SIZE to y > Y_SIZE * 0.75 TO BE MORE STRICT

          if ymin == 0 or y > Y_SIZE * y_camera_threshold or xmin == 0 or xmax == X_SIZE:
            ball = False

          # We make sure all crops are 224x224 (this won't apply if code above is implemented)
          # if ymin == 0:
          #   ymax = CROP_SIZE # = 224
          # elif ymax == Y_SIZE:
          #   ymin = Y_SIZE - CROP_SIZE # = 1024 - 224
          # if xmin == 0:
          #   xmax = CROP_SIZE # == 224 
          # elif xmax == X_SIZE:
          #   xmin = X_SIZE - CROP_SIZE # = 1928 - 224

          if ball_close_to_person:
            arr = np.array(detections)
            persons = arr[arr[:, 0] == PERSON_CLASS]
            xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
            xy_ball = np.array(detection[X_INDEX:Y_INDEX + 1]).astype(float)
            xy_closest_person = xy_persons[spatial.KDTree(xy_persons).query(xy_ball)[1]] # get closest person
            distance, index = spatial.KDTree(xy_persons).query(xy_ball) # get closest distance ball-person
            if distance > ball_person_threshold:
              ball = False

          if ball:
            for r in range(frame - 3, frame): # 3: amount of frames per event
              img = cv2.imread(f'{FOLDER_FRAMES}{video}/{event_id}_{r}.jpg')
              cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
              cv2.imwrite(f'{POSITIVES_FOLDER}crops/{video}/{frame_id}_{r}cropped.jpg', cropped_image)
            break # breaks loop of detections in a single frame (so there are 3 per frame only)

# I ADDED THE FOLLOWING IF STATEMENT:
        # if ball:
        #   break

      if ball:      
        break # breaks loop of frame in frames per event (so there are 3 per event only)

      elif frame == FRAMES_PER_EVENT - 3: # We implement crop on main frame (index 17) based on closest two people
        frame_id = f'{event_id}_{FIRST_FRAME_USED - 1}' # -1 so we get main fraim (index 17)
        label_txt = f'{frame_id}.txt'
        with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
          reader = f.read()
        detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"
        arr = np.array(detections)
        persons = arr[arr[:, 0] == '0']
        xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
        dists = cdist(xy_persons, xy_persons)
        dists[dists == 0] = dists.max()
        arg1, arg2 = np.unravel_index(dists.argmin(), dists.shape)
        two_people = xy_persons[[arg1, arg2]]
        frame_center = np.mean(two_people, axis=0)
        x, y = frame_center
        x = int(x * X_SIZE)
        y = int(y * Y_SIZE)
        ymin = max(y - radius, 0)
        ymax = min(y + radius, Y_SIZE)
        xmin = max(x - radius, 0)
        xmax = min(x + radius, X_SIZE)

        for r in range(FIRST_FRAME_USED - 1, FIRST_FRAME_USED + 2): # here we use -1 and + 2 because we are certain it's from 17 (our main frame)
          img = cv2.imread(f'{FOLDER_FRAMES}{video}/{event_id}_{r}.jpg')
          cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
          cv2.imwrite(f'{POSITIVES_FOLDER}crops/{video}/{frame_id}_{r}cropped.jpg', cropped_image)
        break


Streaming output truncated to the last 5000 lines.
cfbe2e94_0_131_2_27
cfbe2e94_0_131_2_28
cfbe2e94_0_131_2_29
cfbe2e94_0_131_2_30
cfbe2e94_0_131_2_31
cfbe2e94_0_131_2_32
cfbe2e94_0_132_2_18
cfbe2e94_0_132_2_19
cfbe2e94_0_132_2_20
cfbe2e94_0_132_2_21
cfbe2e94_0_132_2_22
cfbe2e94_0_132_2_23
cfbe2e94_0_132_2_24
cfbe2e94_0_132_2_25
cfbe2e94_0_132_2_26
cfbe2e94_0_132_2_27
cfbe2e94_0_132_2_28
cfbe2e94_0_132_2_29
cfbe2e94_0_132_2_30
cfbe2e94_0_132_2_31
cfbe2e94_0_132_2_32
cfbe2e94_0_133_1_18
cfbe2e94_0_134_1_18
cfbe2e94_0_134_1_19
cfbe2e94_0_135_1_18
cfbe2e94_0_135_1_19
cfbe2e94_0_135_1_20
cfbe2e94_0_135_1_21
cfbe2e94_0_135_1_22
cfbe2e94_0_136_2_18
cfbe2e94_0_136_2_19
cfbe2e94_0_136_2_20
cfbe2e94_0_136_2_21
cfbe2e94_0_136_2_22
cfbe2e94_0_136_2_23
cfbe2e94_0_137_0_18
cfbe2e94_0_137_0_19
cfbe2e94_0_138_0_18
cfbe2e94_0_138_0_19
cfbe2e94_0_138_0_20
cfbe2e94_0_138_0_21
cfbe2e94_0_138_0_22
cfbe2e94_0_138_0_23
cfbe2e94_0_138_0_24
cfbe2e94_0_138_0_25
cfbe2e94_0_138_0_26
cfbe2e94_0_138_0_27
cfbe2e94_

In [ ]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
os.chdir(POSITIVES_FOLDER)

In [ ]:
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


In [ ]:
# os.chdir('/content/drive/MyDrive/Bundesliga/')

In [ ]:
# ! ls crops

In [236]:
! ls crops/'1606b0e6_0'  | wc -l

1188


In [ ]:
! ls crops/'3c993bd2_1'  | wc -l

1128


In [ ]:
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


First time checked:

In [ ]:
for video_id in video_ids:
  print(video_id)
  ! ls frames_ms2/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
2426
cfbe2e94_0
3040
cfbe2e94_0
3040
ecf251d4_0
648
3c993bd2_0
659
3c993bd2_1
16036
4ffd5986_0
3050
9a97dae4_1
3091
35bd9041_0
636
35bd9041_1
587
407c5a9e_1
670


Second time checked:

In [ ]:
for video_id in video_ids:
  print(video_id)
  ! ls $FOLDER_FRAMES/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
3940
cfbe2e94_0
4549
cfbe2e94_1
1378
ecf251d4_0
9743
3c993bd2_0
1688
3c993bd2_1
16036
4ffd5986_0
4632
9a97dae4_1
5554
35bd9041_0
1466
35bd9041_1
1377
407c5a9e_1
10498


Third time checked:

In [ ]:
for video_id in video_ids:
  print(video_id)
  ! ls $FOLDER_FRAMES/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
4374
cfbe2e94_0
5003
cfbe2e94_1
2888
ecf251d4_0
13510
3c993bd2_0
3364
3c993bd2_1
16036
4ffd5986_0
5090
9a97dae4_1
10185
35bd9041_0
3081
35bd9041_1
2923
407c5a9e_1
12250


Fourth time checked - 12/11/22 1.14 am:

In [ ]:
for video_id in video_ids:
  print(video_id)
  ! ls $FOLDER_FRAMES/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
7823
cfbe2e94_0
8637
cfbe2e94_1
6383
ecf251d4_0
13510
3c993bd2_0
7248
3c993bd2_1
16036
4ffd5986_0
8661
9a97dae4_1
10185
35bd9041_0
7049
35bd9041_1
6522
407c5a9e_1
12250


Fifth time checked - 12/11/22 8.31 am:

In [24]:
for video_id in video_ids:
  print(video_id)
  ! ls $FOLDER_FRAMES/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
9594
cfbe2e94_0
10675
cfbe2e94_1
9975
ecf251d4_0
13510
3c993bd2_0
12247
3c993bd2_1
16036
4ffd5986_0
10570
9a97dae4_1
10185
35bd9041_0
10633
35bd9041_1
12565
407c5a9e_1
12250


In [ ]:
def get_positive_intervals(TABULAR_FILENAME):
    """ get intervals for events from tabular data (train.csv file) """
    
    df = pd.read_csv(TABULAR_FILENAME)
    df['time_ms'] = round(df['time'] * 1000, 0).astype(int)
    # get positive intervals
    df_starts = df[df['event']=='start'].copy()
    df_ends = df[df['event']=='end'].copy()

    df_positive_intervals = pd.DataFrame({'video_id': df_starts['video_id'].values,
                                        'start_time_ms': df_starts['time_ms'].values,
                                        'end_time_ms': df_ends['time_ms'].values})
    return df_positive_intervals
    

def get_frames_events(video_id, tabular_filename, videos_path, output_path, 
                      negatives=False, neg_size=None, npz=True, jpg=False, dim=None):
    """get frames of events for a given video (positive or negative classes),
       save npz and/or jpg files
       resize (optional)
       function takes:
            video_id
            tabular_filename: filename for tabular data
            videos_path: path to folder where the relevant video is located
            output_path: path to folder where npz and/or jpg files will be saved
            negatives: whether we are taking samples of negative class. 
                If True, we take only negatives, not positives. If False, we take only positives, not negatives.
            neg_size: size of sample of negatives
            npz: whether to save an npz file with results
            jpg: whether to save jpg files
            dim: dimensions of resized images (optional)
    """

    os.chdir(ROOT)
    # get times from tabular data
    if negatives:
        df_positive_intervals = get_positive_intervals(tabular_filename)
        times = get_neg_times(df_positive_intervals, video_id, neg_size)
    else:
        times = get_event_times(tabular_filename)
    n_frames = len(times)

    # capture video
    videoname = ''.join([video_id, '.mp4'])
    cap = cv2.VideoCapture(os.path.join(videos_path, videoname))

    # initialize result list
    res = []

    for i, time in enumerate(tqdm(times)):
        cap.set(cv2.CAP_PROP_POS_MSEC, time)  # move the time
        success, image = cap.read()
        if not success:
            print(f'{i+1} of {n_frames}, video_id: {video_id}, time: {time}, failed to read.')
        # resize
        if dim:
            image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        if npz:
            res.append(image)
        if jpg:
            cv2.imwrite(os.path.join(output_path, f'{video_id}_{time}.jpg'), image)
        del image
    cap.release()
    if npz:
        np.savez_compressed(os.path.join(output_path, ''.join([video_id, '.npz'])), np.array(res))

    if negatives:
        return times

We use persons:

In [ ]:
# for i, frame in enumerate(filenames, 1): # frame of video (filenames contains frames of a single video)
#   label_txt = f'{video}_{i}.txt'
#   with open(f'{FOLDER_LABELS}{label_txt}') as f:
#     reader = f.read()
#   objects = [i.split(' ') for i in reader.split('\n')]
#   ball = False
#   counter = 0

#   for object_ in objects:
#     if object_[CLASS_INDEX] == BALL_CLASS:
#       ball = True
#       x = int(float(object_[X_INDEX]) * X_SIZE)
#       y = int(float(object_[Y_INDEX]) * Y_SIZE)
#       ymin = max(y - radius, 0)
#       ymax = min(y + radius, Y_SIZE)
#       xmin = max(x - radius, 0)
#       xmax = min(x + radius, X_SIZE)

#       img = cv2.imread(frame + '.jpg') # IT CROPS MORE THAN 1 WHEN 2 OR MORE BALLS ARE IN A FRAME, BUT IT OVERWRITES IT
#       cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
#       cv2.imwrite(f'{FOLDER_FRAMES}{frame}_cropped.jpg', cropped_image)
    
#   if not ball and USE_PERSONS:
#     counter += 1
#   if counter == USE_PERSONS_THRESH: 
#     # Maybe create a counter += 1, and when the ball is not detected in 15? consecutive frames, generate the location like this:
#     FIRST_FILENAME_INDEX = 1
#     label_txt = f'{video}_{FIRST_FILENAME_INDEX}.txt'
#     with open(f'{FOLDER_LABELS}{label_txt}') as f: # Actually the reading should have happened before, so I may have to check this.
#       reader = f.read()
#     objects = [i.split(' ') for i in reader.split('\n')][:-1]
#     arr = np.array(objects)
#     persons = arr[arr[:, 0] == '0']
#     xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
#     dists = cdist(xy_persons, xy_persons)
#     dists[dists == 0] = dists.max()
#     arg1, arg2 = np.unravel_index(dists.argmin(), dists.shape)
#     two_people = xy_persons[[arg1, arg2]]
#     frame_center = np.mean(two_people, axis=0)

NameError: ignored